In [2]:
!pip install d2l==1.0.0-beta0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.6/141.6 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 15.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.6/155.6 KB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 KB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 32.8 MB/s eta 0:00:00
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616825 sha256=d7f04ae2fa45bd985e0c091cda2bbabde69bfddf60e0e6ee289b8a217edee930
  Stored in directory: /root/.cache/pip/wheels/27/6d/b3/a3a6e10704795c9b9000f1ab2dc480dfe7bed42f5972806e73
Successfully built gym
  Attempting uninstal

Một trong những bộ dữ liệu được sử dụng rộng rãi để phân loại hình ảnh là bộ dữ liệu MNIST ( LeCun et al. , 1998 ) gồm các chữ số viết tay. Vào thời điểm phát hành vào những năm 1990, nó đã đặt ra một thách thức ghê gớm đối với hầu hết các thuật toán học máy, bao gồm 60.000 hình ảnh của $28\mathbf{x}28$
độ phân giải pixel (cộng với bộ dữ liệu thử nghiệm gồm 10.000 hình ảnh). Nói một cách dễ hiểu, vào thời điểm đó, Sun SPARCStation 5 với RAM khổng lồ 64 MB và 5 MFLOP khổng lồ được coi là thiết bị tiên tiến nhất dành cho máy học tại Phòng thí nghiệm AT&T Bell vào năm 1995. Đạt được độ chính xác cao trong nhận dạng chữ số là một thành phần quan trọng trong việc tự động phân loại thư cho USPS vào những năm 1990. Các mạng sâu như LeNet-5 ( LeCun et al. , 1995 ) , hỗ trợ các máy vectơ có tính bất biến ( Schölkopf et al. , 1996 ) và phân loại khoảng cách tiếp tuyến ( Simard et al. , 1998 )tất cả đều được phép đạt tỷ lệ lỗi dưới 1%.

Trong hơn một thập kỷ, MNIST đóng vai trò là điểm tham chiếu để so sánh các thuật toán học máy. Mặc dù nó hoạt động tốt dưới dạng tập dữ liệu chuẩn, nhưng ngay cả các mô hình đơn giản theo tiêu chuẩn ngày nay cũng đạt được độ chính xác phân loại trên 95%, khiến nó không phù hợp để phân biệt giữa các mô hình mạnh hơn và các mô hình yếu hơn. Hơn nữa, bộ dữ liệu cho phép mức độ chính xác rất cao, thường không thấy trong nhiều bài toán phân loại. Sự phát triển thuật toán sai lệch này hướng tới các họ thuật toán cụ thể có thể tận dụng các tập dữ liệu sạch, chẳng hạn như các phương pháp tập hoạt động và thuật toán tập hoạt động tìm kiếm ranh giới. Ngày nay, MNIST đóng vai trò kiểm tra độ tỉnh táo nhiều hơn là điểm chuẩn. ImageNet ( Deng et al. , 2009 )đặt ra một thách thức phù hợp hơn nhiều. Thật không may, ImageNet quá lớn đối với nhiều ví dụ và hình minh họa trong cuốn sách này, vì sẽ mất quá nhiều thời gian để đào tạo để làm cho các ví dụ tương tác. Thay vào đó, chúng tôi sẽ tập trung thảo luận trong các phần sắp tới về bộ dữ liệu Fashion-MNIST tương tự về chất lượng nhưng nhỏ hơn nhiều ( Xiao et al. , 2017 ), được phát hành vào năm 2017. Nó chứa hình ảnh của 10 loại quần áo tại 
độ phân giải pixel.

In [3]:
%matplotlib inline
import time
import torch
import torchvision
from torchvision import transforms
from d2l import torch as d2l

d2l.use_svg_display()

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


#4.2.1. Data Loading

Vì đây là tập dữ liệu được sử dụng thường xuyên nên tất cả các khung chính đều cung cấp các phiên bản đã được xử lý trước của nó. Chúng tôi có thể tải xuống và đọc bộ dữ liệu Fashion-MNIST vào bộ nhớ bằng các tiện ích khung tích hợp sẵn.

In [4]:
class FashionMNIST(d2l.DataModule):
  def __init__(self, batch_size = 64, resize = (28, 28)):
    super().__init__()
    self.save_hyperparameters()
    trans = transforms.Compose([transforms.Resize(resize),
                                transforms.ToTensor()])
    self.train = torchvision.datasets.FashionMNIST(
        root=self.root, train = True, transform=trans, download = True)
    self.val = torchvision.datasets.FashionMNIST(
        root=self.root, train = False, transform=trans, download = True)



Fashion-MNIST bao gồm các hình ảnh từ 10 danh mục, mỗi danh mục được biểu thị bằng 6.000 hình ảnh trong tập dữ liệu huấn luyện và 1.000 trong tập dữ liệu thử nghiệm. Tập dữ liệu thử nghiệm được sử dụng để đánh giá hiệu suất của mô hình (không được sử dụng tập dữ liệu này để đào tạo). Do đó, tập huấn luyện và tập kiểm tra lần lượt chứa 60.000 và 10.000 hình ảnh.

In [5]:
data = FashionMNIST(resize = (32,32))
len(data.train), len(data.val)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ../data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ../data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ../data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ../data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ../data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ../data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ../data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/FashionMNIST/raw



(60000, 10000)

Các hình ảnh có thang độ xám và được nâng cấp lên 32x32 pixel ở độ phân giải trên. Điều này tương tự với bộ dữ liệu MNIST ban đầu bao gồm các hình ảnh đen trắng (nhị phân). Tuy nhiên, xin lưu ý rằng hầu hết dữ liệu hình ảnh hiện đại có 3 kênh (đỏ, lục, lam) và hình ảnh siêu phổ có thể có hơn 100 kênh (cảm biến HyMap có 126 kênh). Theo quy ước, chúng tôi lưu trữ hình ảnh dưới dạng $c * h * w$ tensor, với $c$
là số kênh màu, $h$ là chiều cao và
$w$ là chiều rộng.

In [6]:
data.train[0][0].shape

torch.Size([1, 32, 32])

Các danh mục của Fashion-MNIST có tên dễ hiểu với con người. Phương thức tiện lợi sau đây chuyển đổi giữa các nhãn số và tên của chúng.

In [7]:
@d2l.add_to_class(FashionMNIST)
def text_labels(self, indices):
  labels = ['t-shirt', 'trouser', 'pullover', 'dress', 'coat',
              'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']
  return [labels[int(i)] for i in indices]

#4.2.2. Đọc một Minibatch

Để làm cho cuộc sống của chúng tôi dễ dàng hơn khi đọc từ các tập huấn luyện và kiểm tra, chúng tôi sử dụng trình lặp dữ liệu tích hợp thay vì tạo từ đầu. Hãy nhớ lại rằng tại mỗi lần lặp, trình lặp dữ liệu đọc một gói nhỏ dữ liệu có kích thước batch_size. Chúng tôi cũng xáo trộn ngẫu nhiên các ví dụ cho trình lặp dữ liệu huấn luyện.

In [8]:
@d2l.add_to_class(FashionMNIST)  
def get_dataloader(self, train):
    data = self.train if train else self.val
    return torch.utils.data.DataLoader(data, self.batch_size, shuffle=train,
                                       num_workers=self.num_workers)

Để xem nó hoạt động như thế nào, hãy tải một loạt hình ảnh nhỏ bằng cách gọi `train_dataloader` phương thức. Nó chứa 64 hình ảnh.

In [9]:
X, y = next(iter(data.train_dataloader()))
print(X.shape, X.dtype, y.shape, y.dtype)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


torch.Size([64, 1, 32, 32]) torch.float32 torch.Size([64]) torch.int64


Hãy nhìn vào thời gian cần thiết để đọc các hình ảnh. Mặc dù nó là một trình tải tích hợp, nhưng nó không quá nhanh. Tuy nhiên, điều này là đủ vì việc xử lý hình ảnh với mạng sâu sẽ lâu hơn một chút. Do đó, việc đào tạo một mạng sẽ không bị hạn chế IO là đủ tốt.

In [10]:
tic = time.time()
for X, y in data.train_dataloader():
    continue
f'{time.time() - tic:.2f} sec'

'17.23 sec'

# 4.2.3. Visualization

Chúng tôi sẽ sử dụng bộ dữ liệu Fashion-MNIST khá thường xuyên. Một chức năng tiện lợi show_imagescó thể được sử dụng để trực quan hóa hình ảnh và các nhãn liên quan. Chi tiết về việc thực hiện nó được chuyển đến phần phụ lục.

In [11]:
def show_images(imgs, num_rows, num_cols, titles=None, scale=1.5): 
    """Plot a list of images."""
    raise NotImplementedError

Hãy đưa nó vào sử dụng tốt. Nói chung, bạn nên trực quan hóa và kiểm tra dữ liệu mà bạn đang đào tạo. Con người rất giỏi trong việc phát hiện ra những khía cạnh bất thường và do đó, trực quan hóa đóng vai trò là biện pháp bảo vệ bổ sung chống lại những sai sót và sai sót trong thiết kế thí nghiệm. Dưới đây là các hình ảnh và nhãn tương ứng của chúng (ở dạng văn bản) cho một vài ví dụ đầu tiên trong tập dữ liệu huấn luyện.

In [ ]:
@d2l.add_to_class(FashionMNIST)  
def visualize(self, batch, nrows=1, ncols=8, labels=[]):
    X, y = batch
    if not labels:
        labels = self.text_labels(y)
    d2l.show_images(X.squeeze(1), nrows, ncols, titles=labels)
batch = next(iter(data.val_dataloader()))
# Y_hat = batch[:-1]
# print(Y_hat[0].shape[-1])
# Y_hat[0] = Y_hat[0].reshape((-1, Y_hat[0].shape[-1]))
# print(Y_hat[0].shape)
data.visualize(batch)

